Some Data Loading/Validating/Cleaning Basics (iPython Notebook)
=======================================================

In [ ]:
# load most relevant packages: NumPy, Pandas and some plotting libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Load the Task Data

Load the `02_task_data.csv` dataset (data description in task_data_description.txt).

In [ ]:
# display content of the cufrent directory (use "ls" on Unix based systems!)
#!dir

In [ ]:
# display content of txt file (use "cat" on Unix based systems!)
!type 02_task_data_description.txt

In [ ]:
# Unix only: show header of the CSV file
#!head 02_task_data.csv

In [ ]:
df = pd.read_csv('02_task_data.csv', index_col='Id')
df.info()

In [ ]:
df.head()

## Data Types and Measurement Scales

Select at least 2 features with nominal scale, 1 with dichotomous scale, 2 with ordinal scale, 2 with discrete numeric scale, and 2 with numeric continuous scale.

### Nominal Scale

1. Neighborhood: Physical locations within Ames city limits
    - Blmngtn Bloomington Heights
    - Blueste Bluestem
    - BrDale Briardale
    - BrkSide Brookside
    - ClearCr Clear Creek
    - CollgCr College Creek
    - Crawfor Crawford
    - Edwards Edwards
    - Gilbert Gilbert
    - IDOTRR Iowa DOT and Rail Road
    - MeadowV Meadow Village
    - Mitchel Mitchell
    - Names North Ames
    - NoRidge Northridge
    - NPkVill Northpark Villa
    - NridgHt Northridge Heights
    - NWAmes Northwest Ames
    - OldTown Old Town
    - SWISU South & West of Iowa State University
    - Sawyer Sawyer
    - SawyerW Sawyer West
    - Somerst Somerset
    - StoneBr Stone Brook
    - Timber Timberland
    - Veenker Veenker
2. MSZoning: Identifies the general zoning classification of the sale.
    - A Agriculture
    - C Commercial
    - FV Floating Village Residential
    - I Industrial
    - RH Residential High Density
    - RL Residential Low Density
    - RP Residential Low Density Park
    - RM Residential Medium Density

### Dichotomous Scale

1. CentralAir: Central air conditioning
    - N No
    - Y Yes

### Ordinal Scale

1. LotShape: General shape of property
    - Reg Regular
    - IR1 Slightly irregular
    - IR2 Moderately Irregular
    - IR3 Irregular
2. Utilities: Type of utilities available
    - AllPub All public Utilities (E,G,W,& S)	
    - NoSewr Electricity, Gas, and Water (Septic Tank)
    - NoSeWa Electricity and Gas Only
    - ELO Electricity only

### Discrete Numeric Scale

1. Fireplaces: Number of fireplaces
2. YearBuilt: Original construction date

### Numeric Continuous Scale

1. LotFrontage: Linear feet of street connected to property
2. LotArea: Lot size in square feet

## Domains Validation

Validate domains of all features selected in previous step.

In [ ]:
features = ['Neighborhood', 'MSZoning', 'CentralAir', 'LotShape', 'Utilities', 'Fireplaces',
            'YearBuilt', 'LotFrontage', 'LotArea']

In [ ]:
for feature in features:
    print(feature, df[feature].unique())

All values follow the data description file.
There are only minor but consistent differences.
For example 'C' in data description is 'C (all)' in data.
Also some cases differ e.g. 'NAmes' vs. 'Names'.

In [ ]:
for feature in ['YearBuilt', 'LotFrontage', 'LotArea']:
    print(df[feature].describe(), '\n')

Numerical values are also in valid domains.

## Format Conversion

Convert them to proper format.

In [ ]:
df['Neighborhood'] = df['Neighborhood'].astype('category', ordered=True)
df['MSZoning'] = df['MSZoning'].astype('category', ordered=True)
df['CentralAir'] = df['CentralAir'].astype('category', ordered=True)
df['LotShape'] = df['LotShape'].astype('category', ordered=True, categories=['Reg', 'IR1', 'IR2', 'IR3'])
df['Utilities'] = df['Utilities'].astype('category', ordered=True, categories=['AllPub', 'NoSewr', 'NoSeWa', 'ELO'])
df['LotArea'] = df['LotArea'].astype('float64')

In [ ]:
df[features].dtypes

## Outliers Detection

Try to detect some outliers (use GrLivArea and SalePrice).

In [ ]:
# look at the boxplots of both variables (dispersion of data)
fig, (ax1, ax2) = plt.subplots(ncols=2)
df['GrLivArea'].plot.box(ax=ax1)
df['SalePrice'].plot.box(ax=ax2)
fig.tight_layout()

How to interprete a boxplot?
* min/max values
* median, 2nd quartile (25%), 3rd quartile (75%)
* whiskers are at +/-1.5 of the IQR (interquartile range) - everything outside this is an outlier

In [ ]:
# draw a scatterplot
sns.jointplot(x='GrLivArea', y='SalePrice', data=df)

Plot of two variables; useful for examining/visualizing dependencies between these variables

In [ ]:
# Plot the distribution with a histogram and maximum likelihood gaussian distribution fit (for live area)
sns.distplot(df['GrLivArea'], rug=True)

In [ ]:
# Plot the distribution with a histogram and maximum likelihood gaussian distribution fit (for sale price)
sns.distplot(df['SalePrice'], rug=True)

In [ ]:
df[df['SalePrice'] > 700000]

I detected the four samples below as outliers. Because they have much higher GrLivArea
(Above grade (ground) living area square feet).
Two of them are abnormally expesive and the other two were sold when not fully completed.

On the other hand. The two with high sale price follow the trend which shows the scatter plot.
Correlation between GrLivArea and SalePrice is 0.71 (shown in the scatter plot).
And if the two other were sold when completed they might be expensive.

In [ ]:
with pd.option_context('display.max_columns', None):
    print(df[df['GrLivArea'] > 4000])

## Missing Data

Determine all features with missing data.
Find a feature that should be completely droped from further analysis.
Select a feature with reasonable small portion of missing values (but with at least 5) and try to fill it properly.

In [ ]:
# features with missing data
fearutres_with_nan = df.columns[df.isnull().any()].tolist()
print('Number of samples:', df.shape[0])
df[fearutres_with_nan].isnull().sum().sort_values(ascending=False)

### Dropped Feature

Feature that should be completely droped from further analysis: LotFrontage (Linear feet of street connected to property)

It might seem that PoolQC, MiscFeature, Alley, Fence, FireplaceQu features have more missing values but in these cases missing values ussualy mean that the feature is not available (e.g. Fence is not available).

In [ ]:
del df['LotFrontage']

### Filling `MasVnrType` Feature

In [ ]:
# apply operation to all rows of a column (here: multiply by 100)
df['MasVnrType'].value_counts(normalize=True).mul(100)

As almost 60% of MasVnrType is of value None it's reasonable to claim that the missing values might be set to None. Then MasVnrArea might be also set to 0.

In [ ]:
df.loc[df['MasVnrType'].isnull(), ['MasVnrType', 'MasVnrArea']] = ('None', 0)
df[['MasVnrType', 'MasVnrArea']].isnull().sum()

### Exercise 2:
#### (1) Load the data-set '02_dow_jones_index.data' using pandas ('02_dow_jones_index.names' contains a description of the data).
#### (2) Validate the numerical columns in the following ways: (a) list all values of the columns with non-numerical values and their occurrences (use a dictionary); (b) convert share prices to floats (spoiler alert: df\['open'\] = df\['open'\].str.strip('$').astype('float64') ); (c) look at the dispersion of the columns with numerical values (7-number-summary and/or boxplot)
#### (3) Check if all rows are in the proper chronological order and fix the order if necessary. Correct the date (spoiler alert: df\['date_corr'\] = pd.to_datetime(df\['date'\]) ) and plot the chart of the closing stock price for the Cisco share.
#### (4) Add a column with the absolute difference between highest and lowest stock price of this day.